In [1]:
import tensorflow as tf
from pathlib import Path
from os.path import abspath

available_gpus = tf.config.list_physical_devices('GPU')
print(f"Number of available GPUs: {len(available_gpus)}")

try:
    tf.config.experimental.set_memory_growth(available_gpus[0], True)
except:
    pass


Number of available GPUs: 1


# Parameters

In [2]:
BATCH_SIZE = 256
EPOCHS = 50
TEST_SPLIT = 0.2
VALID_SPLIT = 0.1
SEED = 7321
VOCABULARY_SIZE = 100_000
MAX_SEQUENCE_LEN = 250
EMBEDDING_DIM = 75

# Loading dataset

In [3]:
SRC_DIR = Path(abspath('')).parent.parent
RAW_DIR = SRC_DIR / "raw"
DATASET_DIR = str((RAW_DIR / "dataset").resolve())
BEST_MODEL_DIR = SRC_DIR / "models" / "checkpoints"
TENSOR_BOARD_DIR = SRC_DIR / "models" / "tensor_board"

train_ds, test_ds = tf.keras.utils.text_dataset_from_directory(
    directory=DATASET_DIR,
    label_mode='categorical',
    validation_split=TEST_SPLIT,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED,
    subset='both',
)
val_ds_size = int(test_ds.cardinality().numpy() * VALID_SPLIT)
val_ds = test_ds.take(val_ds_size)
test_ds = test_ds.skip(val_ds_size)

Found 1438 files belonging to 3 classes.
Using 1151 files for training.
Using 287 files for validation.
Metal device set to: Apple M1 Max


2022-11-21 21:13:03.473396: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-21 21:13:03.473564: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Instantiate words vectorizing layer

In [4]:
train_text = train_ds.map(lambda x, y: x)

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=VOCABULARY_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LEN
)
vectorize_layer.adapt(train_text, batch_size=BATCH_SIZE)

2022-11-21 21:13:06.843974: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-21 21:13:06.875174: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


# Batch, cache, autotune

In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1500).prefetch(AUTOTUNE)
test_ds = test_ds.cache().shuffle(1500).prefetch(AUTOTUNE)
val_ds = val_ds.cache().shuffle(1500).prefetch(AUTOTUNE)

list(train_ds.as_numpy_iterator())
list(test_ds.as_numpy_iterator())
list(val_ds.as_numpy_iterator())
print("Dataset cached successfully!")

Dataset cached successfully!


# Build model

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    tf.keras.layers.Embedding(input_dim=VOCABULARY_SIZE, output_dim=EMBEDDING_DIM),
    tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 250)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 250, 75)           7500000   
                                                                 
 lstm (LSTM)                 (None, 100)               70400     
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                                 
Total params: 7,570,703
Trainable params: 7,570,703
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [7]:
import tensorflow_addons as tfa

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[
        'accuracy',
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.AUC(),
        tfa.metrics.F1Score(num_classes=3)
    ]
)

## Model callbacks

In [9]:
BEST_MODEL_FILE = BEST_MODEL_DIR / "lstm_best3"

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6, verbose=1, min_delta=0.0001)
save_best = tf.keras.callbacks.ModelCheckpoint(filepath=BEST_MODEL_FILE, monitor='loss', save_best_only=True)
csv_logger = tf.keras.callbacks.CSVLogger(filename=BEST_MODEL_FILE/"history_3.csv")
# tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=str(TENSOR_BOARD_DIR.resolve()))

# Fit the model

In [10]:
with tf.device('/GPU:0'):
    history = model.fit(
        train_ds,
        epochs=EPOCHS,
        validation_data=val_ds,
        callbacks=[save_best, early_stop, csv_logger],
    )

Epoch 1/50


2022-11-21 21:14:54.288278: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 748s 151s/step - loss: 1.0944 - accuracy: 0.3710 - recall: 0.0000e+00 - precision: 0.0000e+00 - auc: 0.5345 - f1_score: 0.3246
Epoch 2/50
5/5 [==============================] - 756s 151s/step - loss: 1.0683 - accuracy: 0.5030 - recall: 0.0000e+00 - precision: 0.0000e+00 - auc: 0.6584 - f1_score: 0.3931
Epoch 3/50
5/5 [==============================] - 758s 152s/step - loss: 1.0342 - accuracy: 0.5065 - recall: 0.0938 - precision: 0.5070 - auc: 0.7079 - f1_score: 0.4012
Epoch 4/50
5/5 [==============================] - 774s 155s/step - loss: 0.9901 - accuracy: 0.5621 - recall: 0.1234 - precision: 0.5279 - auc: 0.7722 - f1_score: 0.5728
Epoch 5/50
5/5 [==============================] - 784s 157s/step - loss: 0.8480 - accuracy: 0.6777 - recall: 0.3501 - precision: 0.6796 - auc: 0.8228 - f1_score: 0.6747
Epoch 6/50
5/5 [==============================] - 775s 155s/step - loss: 0.6993 - accuracy: 0.6759 - recall: 0.6568 - precision: 0.6799 - auc: 0.8602 

# Evaluate the model

In [12]:
model.evaluate(test_ds)

2022-11-22 07:55:40.813678: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 6s 3s/step - loss: 0.2478 - accuracy: 0.9477 - recall: 0.9477 - precision: 0.9477 - auc: 0.9780 - f1_score: 0.9460


[0.24784789979457855,
 0.9477351903915405,
 0.9477351903915405,
 0.9477351903915405,
 0.9780378341674805,
 array([0.93548393, 0.93023264, 0.97222227], dtype=float32)]